# GabeDA Features (Customer Profile - Monthly Snapshot)

This notebook creates customer behavior profiles based on current month activity.
It aggregates all transactions per customer for the current month to create a monthly snapshot.

**Input:** Preprocessed transactions from 01_transactions notebook  
**Output:** Customer profiles (1 row per customer)  
**Group By:** `customer_id`

## 1. Setup: Imports, Context Loading, Logging

## 0. Project Root Setup (Auto-generated)

In [1]:
# Auto-detect project root and add to Python path
import os
import sys
from pathlib import Path

# Get the project root (2 levels up from notebooks/development or notebooks/from_store)
notebook_dir = Path.cwd() if '__file__' not in globals() else Path(__file__).parent
project_root = notebook_dir.parent.parent

# Change to project root
os.chdir(project_root)

# Add project root to Python path if not already there
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

print(f"Working directory: {os.getcwd()}")
print(f"Project root: {project_root}")

Working directory: c:\Projects\play\khujta_ai_business
Project root: c:\Projects\play\khujta_ai_business


In [2]:
import pandas as pd
import numpy as np

# v2.0 Refactored imports
from src.utils.logger import setup_logging, get_logger
from src.core.context import GabedaContext
from src.core.persistence import load_context_state, get_latest_state, save_context_state
from src.core.constants import *
from src.features.store import FeatureStore
from src.features.resolver import DependencyResolver
from src.features.detector import FeatureTypeDetector
from src.features.analyzer import FeatureAnalyzer
from src.execution.calculator import FeatureCalculator
from src.execution.groupby import GroupByProcessor
from src.execution.executor import ModelExecutor
from src.export.excel import ExcelExporter

# Load latest context state
client_name = 'test_client'
latest_state = get_latest_state(client_name, base_dir='data/context_states')

if latest_state:
    ctx, base_cfg = load_context_state(latest_state)
    print(f"✓ Loaded latest state: {latest_state}")
else:
    raise FileNotFoundError(f"No context state found for client '{client_name}'")

# Setup logging
setup_logging(log_level=base_cfg.get('log_level', 'INFO'), 
              config={'client': base_cfg.get('client', 'unknown_client')})
logger = get_logger(__name__)

print(f"\n✓ Context loaded successfully!")
print(f"  - Original run_id: {ctx.original_run_id}")
print(f"  - New run_id: {ctx.run_id}")
print(f"  - Available datasets: {len(ctx.list_datasets())} datasets")

✓ Loaded latest state: data\context_states\test_client_20251022_150907
📝 Run instance ID: test_client_20251022_151134 - Logging [INFO] to: logs\test_client_20251022_151134.log

✓ Context loaded successfully!
  - Original run_id: test_client_20251022_151119
  - New run_id: test_client_20251022_151134
  - Available datasets: 17 datasets


## 2. Load Input Data

In [3]:
# Get input dataset
input_df = ctx.get_dataset('transactions_filters')

print(f"✓ Input dataset loaded")
print(f"  - Shape: {input_df.shape}")
print(f"  - Date range: {input_df['dt_date'].min()} to {input_df['dt_date'].max()}")
print(f"  - Unique customers: {input_df['customer_id'].nunique()}")
print(f"\nFirst few rows:")
input_df.head()

✓ Input dataset loaded
  - Shape: (609, 59)
  - Date range: 20251001 to 20251030
  - Unique customers: 15

First few rows:


,in_dt,in_product_id,in_quantity,in_price_total,in_trans_type,in_customer_id,in_description,in_category,in_unit_type,in_stock,...,cost_unit,cost_total,price_unit,price_total,margin_unit,margin_unit_pct,margin_unit_valid,margin_total,margin_total_pct,margin_total_valid
0,2025-10-01 01:02:00,prod8,2.0,52964.0,return,client13,product 8,category B,pack,61.0,...,18792.0,37585.0,26482.0,52964.0,7690.0,29.04,True,15379.0,29.04,True
1,2025-10-01 06:24:00,prod4,6.0,177195.0,sale,client6,product 4,category B,unit,30.0,...,21526.0,129155.0,29533.0,177195.0,8007.0,27.11,True,48040.0,27.11,True
2,2025-10-01 08:38:00,prod7,2.0,70492.0,return,client12,product 7,category A,unit,78.0,...,25754.0,51509.0,35246.0,70492.0,9492.0,26.93,True,18983.0,26.93,True
3,2025-10-01 09:59:00,prod2,4.0,86751.0,sale,client3,product 2,category A,unit,80.0,...,12947.0,51786.0,21688.0,86751.0,8741.0,40.30,True,34965.0,40.31,True
4,2025-10-01 10:07:00,prod3,3.0,76465.0,sale,client12,product 3,category B,unit,47.0,...,16943.0,5083.0,25488.0,76465.0,8545.0,33.53,True,71382.0,93.35,True


## 3. Define Features

Customer behavior metrics for monthly snapshot:  
- Spending patterns (total spent, transaction count, average order value)
- Purchase behavior (items purchased, favorite products)
- Preferences (preferred shopping day, preferred shopping time)

In [4]:
# ===== Customer Behavior Profile (Monthly Snapshot) =====
# Based on specs: docs/specs/model/aggr_arch_single_month.md - Dataset 2.4

def month_total_spent(price_total):
    """
    Total amount spent by customer in current month.
    Formula: SUM(price_total)
    """
    return np.sum(price_total)

def month_transaction_count(trans_id):
    """
    Number of transactions by customer in current month.
    Formula: COUNT(DISTINCT trans_id)
    """
    return len(np.unique(trans_id))

def month_items_purchased(quantity):
    """
    Total items purchased by customer in current month.
    Formula: SUM(quantity)
    """
    return np.sum(quantity)

def average_order_value(price_total, trans_id):
    """
    Average spend per transaction for this month.
    Formula: month_total_spent / month_transaction_count
    """
    total_spent = np.sum(price_total)
    transaction_count = len(np.unique(trans_id))
    
    if transaction_count == 0:
        return DEFAULT_FLOAT
    
    return round(total_spent / transaction_count, 2)

def favorite_products(product_id, quantity):
    """
    Top 3 products purchased by customer this month.
    Formula: ARRAY_AGG(product_id ORDER BY quantity DESC LIMIT 3)
    
    Returns: Comma-separated string of top 3 product IDs
    """
    # Aggregate quantity by product
    product_totals = {}
    for pid, qty in zip(product_id, quantity):
        product_totals[pid] = product_totals.get(pid, 0) + qty
    
    # Sort by quantity and get top 3
    top_products = sorted(product_totals.items(), key=lambda x: x[1], reverse=True)[:3]
    
    # Return as comma-separated string
    return ','.join([pid for pid, qty in top_products])

def preferred_shopping_day(dt_weekday_name):
    """
    Most common shopping day of the week for this customer.
    Formula: MODE(weekday_name)
    
    Returns: Most frequent weekday name (e.g., 'Monday', 'Friday')
    """
    if len(dt_weekday_name) == 0:
        return 'Unknown'
    
    # Count occurrences of each weekday
    from collections import Counter
    day_counts = Counter(dt_weekday_name)
    
    # Return most common day
    most_common = day_counts.most_common(1)[0][0]
    return most_common

def preferred_shopping_time(hour):
    """
    Most common time segment when customer shops.
    Formula: MODE(time_segment) based on hour
    
    Returns: 'Morning' (5-12), 'Afternoon' (12-17), 'Evening' (17-21), 'Night' (21-5)
    """
    if len(hour) == 0:
        return 'Unknown'
    
    # Convert hours to time segments
    time_segments = []
    for h in hour:
        if 5 <= h < 12:
            time_segments.append('Morning')
        elif 12 <= h < 17:
            time_segments.append('Afternoon')
        elif 17 <= h < 21:
            time_segments.append('Evening')
        else:
            time_segments.append('Night')
    
    # Find most common segment
    from collections import Counter
    segment_counts = Counter(time_segments)
    
    most_common = segment_counts.most_common(1)[0][0]
    return most_common

print("✓ Feature functions defined: 7 attributes")

✓ Feature functions defined: 7 attributes


## 4. Configure Model

In [5]:
# Collect features into dictionary
features = {
    'month_total_spent': month_total_spent,
    'month_transaction_count': month_transaction_count,
    'month_items_purchased': month_items_purchased,
    'average_order_value': average_order_value,
    'favorite_products': favorite_products,
    'preferred_shopping_day': preferred_shopping_day,
    'preferred_shopping_time': preferred_shopping_time,
}

# Model configuration
cfg_model = {
    'model_name': 'customer_profile',
    'input_dataset_name': 'transactions_filters',
    'group_by': ['customer_id'],  # No time dimension - monthly snapshot per customer
    'row_id': 'in_trans_id',
    'output_cols': list(features.keys()),
    'features': features,
}

print(f"✓ Model configured: '{cfg_model['model_name']}'")
print(f"  - Group by: {cfg_model['group_by']}")
print(f"  - Output features: {len(cfg_model['output_cols'])}")

✓ Model configured: 'customer_profile'
  - Group by: ['customer_id']
  - Output features: 7


## 5. Prepare Features (Store, Resolve Dependencies, Save Config)

In [6]:
# Initialize feature store and store features
feature_store = FeatureStore()
feature_store.store_features(features, model_name=cfg_model['model_name'], auto_save=True)

# Resolve dependencies
resolver = DependencyResolver(feature_store)
in_cols, exec_seq, ext_cols = resolver.resolve_dependencies(
    output_cols=cfg_model['output_cols'],
    available_cols=input_df.columns.tolist(),
    group_by=cfg_model.get('group_by'),
    model=cfg_model['model_name']
)

# Update model config with resolved dependencies
cfg_model['in_cols'] = in_cols
cfg_model['exec_seq'] = exec_seq
cfg_model['ext_cols'] = ext_cols

# Save master configuration
feature_store.save_master_config(
    model_name=cfg_model['model_name'],
    model_config=cfg_model
)

print("✓ Features prepared and dependencies resolved")
print(f"  - Input columns needed: {len(in_cols)}")
print(f"  - Execution sequence: {exec_seq}")
print(f"  - Master config saved: feature_store/{cfg_model['model_name']}/master_cfg.json")

✓ Features prepared and dependencies resolved
  - Input columns needed: 6
  - Execution sequence: ['month_total_spent', 'month_transaction_count', 'month_items_purchased', 'average_order_value', 'favorite_products', 'preferred_shopping_day', 'preferred_shopping_time']
  - Master config saved: feature_store/customer_profile/master_cfg.json


## 6. Execute Model (Initialize Components + Execute + Store Results)

In [7]:
# Initialize execution components
detector = FeatureTypeDetector()
analyzer = FeatureAnalyzer(feature_store, detector)
calculator = FeatureCalculator()
groupby_processor = GroupByProcessor(calculator, detector)
executor = ModelExecutor(analyzer, groupby_processor, context=ctx)

# Execute model
output = executor.execute_model(
    cfg_model=cfg_model,
    input_dataset_name=cfg_model['input_dataset_name']
)

# Store results in context
ctx.set_model_output(cfg_model['model_name'], output, cfg_model)

print("✓ Model executed successfully!")
print(f"  - Filters: {output['filters'].shape if output['filters'] is not None else 'None'}")
print(f"  - Attributes: {output['attrs'].shape if output['attrs'] is not None else 'None'}")
print(f"  - Customers profiled: {output['attrs'].shape[0] if output['attrs'] is not None else 0}")

✓ Model executed successfully!
  - Filters: (609, 59)
  - Attributes: (15, 8)
  - Customers profiled: 15


## 7. View Results

In [8]:
# View customer profiles (aggregated attributes)
attrs = ctx.get_model_attrs(cfg_model['model_name'])
print(f"Customer Profiles (n={len(attrs)}):")
attrs.head(10)

Customer Profiles (n=15):


,customer_id,month_total_spent,month_transaction_count,month_items_purchased,average_order_value,favorite_products,preferred_shopping_day,preferred_shopping_time
0,CLIENT1,3835713.0,43,176,89202.63,"PROD2,PROD9,PROD8",Friday,Afternoon
1,CLIENT10,2642342.0,33,116,80070.97,"PROD9,PROD7,PROD1",Saturday,Afternoon
2,CLIENT11,3263459.0,42,175,77701.40,"PROD9,PROD8,PROD10",Thursday,Afternoon
3,CLIENT12,4057118.0,59,184,68764.71,"PROD1,PROD3,PROD10",Thursday,Afternoon
4,CLIENT13,3407417.0,31,144,109916.68,"PROD8,PROD3,PROD6",Friday,Afternoon
5,CLIENT14,4637137.0,50,198,92742.74,"PROD9,PROD4,PROD10",Wednesday,Afternoon
6,CLIENT15,3423471.0,41,166,83499.29,"PROD10,PROD7,PROD1",Thursday,Afternoon
7,CLIENT2,3425998.0,44,151,77863.59,"PROD2,PROD6,PROD9",Thursday,Afternoon
8,CLIENT3,4731427.0,40,188,118285.68,"PROD7,PROD4,PROD8",Thursday,Afternoon
9,CLIENT4,2597804.0,34,106,76406.00,"PROD10,PROD3,PROD1",Wednesday,Afternoon


In [9]:
# View summary statistics
print("Customer Spending Summary:")
attrs[['month_total_spent', 'month_transaction_count', 'average_order_value']].describe()

Customer Spending Summary:


,month_total_spent,month_transaction_count,average_order_value
count,1.500000e+01,15.000000,15.000000
mean,3.566388e+06,40.600000,88377.004667
std,7.305783e+05,7.414272,13843.930085
min,2.504940e+06,30.000000,68764.710000
25%,3.287843e+06,36.500000,78967.280000
50%,3.423471e+06,40.000000,84928.900000
75%,3.946416e+06,43.500000,90972.685000
max,4.782574e+06,59.000000,118285.680000


In [10]:
# View preferred shopping patterns
print("Shopping Day Preferences:")
print(attrs['preferred_shopping_day'].value_counts())
print("\nShopping Time Preferences:")
print(attrs['preferred_shopping_time'].value_counts())

Shopping Day Preferences:
preferred_shopping_day
Thursday     7
Friday       3
Saturday     2
Wednesday    2
Tuesday      1
Name: count, dtype: int64

Shopping Time Preferences:
preferred_shopping_time
Afternoon    15
Name: count, dtype: int64


## 8. Export to Excel

In [11]:
# Export model results to Excel
exporter = ExcelExporter(ctx)
output_file = f'outputs/{cfg_model["model_name"]}_export.xlsx'
exporter.export_model(cfg_model['model_name'], output_file, include_input=True)

print(f"✓ Export complete: {output_file}")
print("\nExcel tabs:")
print(f"  1. {cfg_model['input_dataset_name']} (input)")
print(f"  2. {cfg_model['model_name']}_filters")
print(f"  3. {cfg_model['model_name']}_attrs")

✓ Export complete: outputs/customer_profile_export.xlsx

Excel tabs:
  1. transactions_filters (input)
  2. customer_profile_filters
  3. customer_profile_attrs


## 9. Save Context State

Save the complete context state for use in downstream notebooks:

In [12]:
# Save context state (datasets, config, metadata)
state_dir = save_context_state(ctx=ctx, base_cfg=base_cfg)

print(f"✓ Context state saved: {state_dir}")
print(f"  - Total datasets: {len(ctx.datasets)}")
print(f"\nTo load this state in another notebook:")
print(f"  from src.core.persistence import load_context_state")
print(f"  ctx, base_cfg = load_context_state('{state_dir}')")

✓ Context state saved: data\context_states\test_client_20251022_150907
  - Total datasets: 19

To load this state in another notebook:
  from src.core.persistence import load_context_state
  ctx, base_cfg = load_context_state('data\context_states\test_client_20251022_150907')
